### 描述器

描述器功能强大，应用广泛，它可以控制我们访问属性、方法的行为，是@property、super、静态方法、类方法、甚至属性、实例背后的实现机制，是一种比较底层的设计。

当类中有大量property时，为了避免代码重复，可以使用描述器。

定义：从描述器的创建来说，一个类中定义了__get__、__set__、__delete__中的一个或几个，这个类的实例就可以叫做一个描述器。

s.name时的访问顺序:

```
- 程序会先查找 s.__dict__['name'] 是否存在
- 不存在再到type(s).__dict__['name']中查找
- 然后找type(s)的父类
- 期间找到的是普通值就输出，如果找到的是一个描述器，则调用__get__方法
```

In [154]:
# A descriptor
class String:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        instance.__dict__[self.name] = value

# A class with a descriptor
class Person:
    # 绑定描述器
    name = String('name')

    def __init__(self, name):
        # 资料描述器同名，优先调用描述器中的set方法
        self.name = name
        
s = Person('Guido')
print(s.name)

Guido


>此外还有特例，与描述器的种类有关

```
- 同时定义了__get__和__set__方法的描述器称为资料描述器
- 只定义了__get__的描述器称为非资料描述器
- 行为上的不同：如果有__set__，定义时就当做描述器，否则，就当做属性，因为没办法调用其中的__set__
```

In [139]:
# 既有__get__又有__set__，是一个资料描述器
class M:
    def __init__(self):
        self.x = 1
        
    def __get__(self, instance, owner):
        print('get m here') # 打印一些信息，看这个方法何时被调用
        return self.x
    
    def __set__(self, instance, value):
        print('set m here') # 打印一些信息，看这个方法何时被调用
        self.x = value + 1 # 这里设置一个+1来更清楚了解调用机制

# 只有__get__是一个非资料描述器
class N:
    def __init__(self):
        self.x = 1
        
    def __get__(self, instance, owner):
        print('get n here') # 打印一些信息，看这个方法何时被调用
        return self.x
        
# 调用描述器的类
class AA:
    m = M() # m就是一个描述器
    n = N()
    def __init__(self, m, n):
        self.m = m # 属性m和描述器m名字相同，调用时发生一些冲突
        self.n = n # 非资料描述器的情况，与m对比
    
aa = AA(2, 5)
print(aa.__dict__) # 只有n没有m, 因为资料描述器同名时，不会访问到属性，会直接访问描述器，所以属性里就查不到m这个属性了
print(AA.__dict__) # m和n都有
print(aa.n) # 5, 非资料描述器同名时调用的是属性，为传入的5
print(AA.n) # 1, 如果是类来访问，就调用的是描述器，返回self.x的值

print(aa.m) # 3, 其实在aa=AA(2,5)创建实例时，进行了属性赋值，其中相当于进行了aa.m=2
print(AA.m) # 3

set m here
{'n': 5}
{'__module__': '__main__', 'm': <__main__.M object at 0x7fb3542a8d90>, 'n': <__main__.N object at 0x7fb3540a1950>, '__init__': <function AA.__init__ at 0x7fb3542ca050>, '__dict__': <attribute '__dict__' of 'AA' objects>, '__weakref__': <attribute '__weakref__' of 'AA' objects>, '__doc__': None}
5
get n here
1
get m here
3
get m here
3


> 实例方法、静态方法和类方法的描述器原理

两种bound method，类方法bound在类上，实例方法bound在实例上

而静态方法实际上是类的函数(function)

In [175]:
class B:
    @classmethod
    def print_classname(cls):
        print('Bob')
        
    @staticmethod
    def print_staticname():
        print('my name is bob')
        
    def print_name(self):
        print('this name')

print(B.__dict__['print_classname']) # 实际上是描述器的类
print(B.print_classname) # bound在类上的方法
print()
print(B.__dict__['print_staticname']) # 实际上是描述器的类
print(b.print_staticname) # 类的函数(function)
print()
print(B.__dict__['print_name']) # 类的函数(function)，其实是描述器绑定到实例里的
print(b.print_name) # bound在实例上的方法
print()

# 得到三个描述器并且调用
B.__dict__['print_classname'].__get__(None, B)() #类实际上定义在描述器里
B.__dict__['print_staticname'].__get__(None, B)() # 静态方法实际上定义在描述器里
B.__dict__['print_name'].__get__(b, B)() # 实例方法，由描述器绑定回实例中

<bound method B.print_classname of <class '__main__.B'>>

<function B.print_staticname at 0x7fb3542a1ef0>

<function B.print_name at 0x7fb35431b830>
<bound method B.print_name of <__main__.B object at 0x7fb354283690>>

Bob
my name is bob
this name


In [7]:
# 绑定方法和函数的区别
import types

class Spam:
    pass

s = Spam()
def bar(self, x):
    print(x)

s.bar = bar
print(s.__dict__) # 函数
s.bar(s, 3) # 函数调用时要给 self

s.bar = types.MethodType(bar, s)
print(s.__dict__) # 绑定方法
s.bar(3) # 绑定方法调用时不用给 self

{'bar': <function bar at 0x7ffe7ca55b00>}
3
{'bar': <bound method bar of <__main__.Spam object at 0x7ffe7ca58910>>}
3


### Property

In [121]:
class Person:
    def __init__(self, first_name):
        # 直接在初始化时调用 set 方法
        self.first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

a = Person('Guido')
print(a.first_name) # Calls the getter
a.first_name = '42' # Calls the setter
print(a.first_name)
# del a.first_name

Guido
42


In [123]:
class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)

    # Getter function
    def get_first_name(self):
        return self._first_name

    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")

    # Make a property from existing get/set methods
    first_name = property(get_first_name, set_first_name, del_first_name)
    
a = Person('Guido')
print(a.first_name) # Calls the getter
a.first_name = '42' # Calls the setter
print(a.first_name)
# del a.first_name

Guido
42


In [114]:
print(Person.first_name.fget)
print(Person.first_name.fset)
print(Person.first_name.fdel)

<function Person.get_first_name at 0x7fb3542d8200>
<function Person.set_first_name at 0x7fb3542d87a0>
<function Person.del_first_name at 0x7fb3542d8050>


一个property其实是 getter、setter 和 deleter 方法的集合，是绑定在类上的变量

子类中重写时：
- 重写所有方法
- 重写部分方法，加父类名称的注释，如 @Person.name.getter

super(SubPerson, SubPerson)方法说明见下一节

In [115]:
# 继承实现
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

# 重写三个方法，且通过类变量调用父类的property
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        print('type', type(self))
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        # 查找父类的类变量，而不是实例变量 super().name
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)
        
s = SubPerson('Guido')
print(s.name)
s.name = 'Larry'

# 只重写getter方法，必须带Person.name.getter，否则会覆盖父类的property
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name
    
s = SubPerson('Guido')
print(s.name)
s.name = 'Larry'

Setting name to Guido
Getting name
type <class '__main__.SubPerson'>
Guido
Setting name to Larry
Getting name
Guido


### 装饰器+描述器的一个例子

在类的装饰器，对类的绑定描述器

In [ ]:
# Descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]

# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


### 描述器做延迟计算

In [181]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [189]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    
#     area = lazyproperty(area) # @lazyproperty 实际上等于这样，定义了一个描述器
#     perimeter = lazyproperty(perimeter)
    
c = Circle(4.0)
print(c.area)
print(c.area) # 第二次访问时，area已经被绑定回实例，此时实例属性优先级高于非资料描述器(即没有set的描述器)

Computing area
50.26548245743669
50.26548245743669


In [197]:
# 优化版本，禁止手动修改area
# 但是所有版本都要绑定到这个property上
# property如果setter留空，则无法set
def lazyproperty(func):
    name = '_lazy_' + func.__name__
    @property
    def lazy(self):
        if hasattr(self, name):
            return getattr(self, name)
        else:
            value = func(self)
            setattr(self, name, value)
            return value

#     @lazy.setter
#     def lazy(self, value):
#         setattr(self, name, value)

    return lazy

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

c = Circle(4.0)
print(c.area)
print(c.area)
# c.area = 25 # AttributeError: can't set attribute

Computing area
50.26548245743669
50.26548245743669


### 描述器做类型约束

描述器、混入类、super() 的使用、类装饰器和元类

In [3]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value


# Descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)


# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)


class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)
        
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

# 混入类
class SizedString(String, MaxSized):
    pass

# 直接定义描述器
class Stock:
    # Specify constraints
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

# 使用类装饰器绑定名称
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate

@check_attributes(name=SizedString(size=8),
                  shares=UnsignedInteger,
                  price=UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

# 使用元类绑定名称
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        # Attach attribute names to the descriptors
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return type.__new__(cls, clsname, bases, methods)

class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [ ]:
# 用装饰器替代混入类
# 实际上使用装饰器来修改描述符类
# 比之前的混入类的方式几乎快100%
# Decorator for applying type checking
def Typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: Typed(expected_type, cls)
    super_set = cls.__set__

    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('expected ' + str(expected_type))
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Decorator for unsigned values
def Unsigned(cls):
    super_set = cls.__set__

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Decorator for allowing sized values
def MaxSized(cls):
    super_init = cls.__init__

    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)

    cls.__init__ = __init__

    super_set = cls.__set__

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Specialized descriptors
@Typed(int)
class Integer(Descriptor):
    pass


@Unsigned
class UnsignedInteger(Integer):
    pass


@Typed(float)
class Float(Descriptor):
    pass


@Unsigned
class UnsignedFloat(Float):
    pass


@Typed(str)
class String(Descriptor):
    pass


@MaxSized
class SizedString(String):
    pass